In [ ]:
!pip install folium==0.10.1

     |████████████████████████████████| 91 kB 3.6 MB/s 
  Attempting uninstall: folium
    Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.10.1 which is incompatible.


In [ ]:
import pandas as pd
import json
import glob
import numpy as np
import math
import folium
from folium import plugins


#from google.colab import files
#uploaded = files.upload()
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
csv_files=glob.glob('/content/drive/MyDrive/Roorkee/*.csv')
from datetime import datetime
from itertools import repeat

for i,file in enumerate(csv_files):
  date_for_file = file.split('_')[-1].split('.')[0]
  date_for_file=date_for_file[:11]+' '+date_for_file[12:]
  date_for_file=date_for_file[:14]+':'+date_for_file[15:]
  date_for_file= pd.to_datetime(date_for_file)
  df = pd.read_csv(file)
  ind=df.shape[0]
  df['Time'] = date_for_file
  df.Time = pd.to_datetime(df.Time, format='%Y-%m-%d %H:%M:%S')
  df['minute'] = df.Time.apply(lambda x: x.hour)*60+df.Time.apply(lambda x: x.minute)
  if i == 0:
   data = pd.DataFrame()
   data = df
  else:
   data = data.append(df)
data.reset_index(inplace = True)

# PM 2.5 Column

for i in enumerate(data):
 pm1 = []
 pm2 =[]
 for j in range(len(data)):
  index = data['realtime'][j].find('PM25')
  val= data['realtime'][j][index+22:index+25]
  pm25 = val.translate({ord(','): None})
  pm1.append(int(pm25))
  index = data['realtime'][j].find('PM10')
  val= data['realtime'][j][index+22:index+25]
  pm10 = val.translate({ord(','): None})
  pm2.append(int(pm10))
 data['PM 2.5'] = pm1
 data['PM 10'] = pm2
result_df=data.sort_values(by='minute')


In [ ]:
freq=[]
k=round(data.shape[0]/(5*ind))
i
for i in range(k-1):
  freq.extend(repeat(i+1,5*ind))
j=round(data.shape[0]/ind)-5*(i+1)
freq.extend(repeat(i+2,j*ind))
#display(freq)
result_df['five_min']=freq
display(result_df)

,index,Unnamed: 0,serialNo,hwid,devicelat,devicelon,devicename,deviceimage,dev_type,dev_category,dev_location_type,location_name,area,city,state,country,pincode,isOnline,last_updated,timezone,realtime,Time,minute,PM 2.5,PM 10,five_min
446,6,6,698712851126,9,29.863076,77.898693,Outdoor 7,https://secure.aqi.in/public/hwimages/Monitor(...,outdoor,office,Office Room,NaN,NaN,Roorkee,Uttarakhand,India,247667.0,0,2021-11-25 01:05:37 AM,Asia/Calcutta,"[{'sensorid': 1, 'sensorname': 'AQI-IN', 'sens...",2021-12-02 10:50:00,650,119,122,1
440,0,0,311181311314,9,29.862951,77.898898,Outdoor 1,https://secure.aqi.in/public/hwimages/Monitor(...,outdoor,office,Office Room,Roorkee,Roorkee,Roorkee,Uttarakhand,India,NaN,0,2021-11-25 07:46:02 AM,Asia/Calcutta,"[{'sensorid': 1, 'sensorname': 'AQI-IN', 'sens...",2021-12-02 10:50:00,650,94,100,1
441,1,1,591671285112,9,29.862980,77.898718,Outdoor 2,https://secure.aqi.in/public/hwimages/Monitor(...,outdoor,office,Office Room,Roorkee,Roorkee,Roorkee,Uttarakhand,India,NaN,0,2021-12-01 09:06:51 PM,Asia/Calcutta,"[{'sensorid': 1, 'sensorname': 'AQI-IN', 'sens...",2021-12-02 10:50:00,650,87,94,1
442,2,2,307531285112,9,29.863096,77.898811,Outdoor 3,https://secure.aqi.in/public/hwimages/Monitor(...,outdoor,office,Office Room,Roorkee,Roorkee,Roorkee,Uttarakhand,India,NaN,0,2021-11-25 06:09:05 AM,Asia/Calcutta,"[{'sensorid': 1, 'sensorname': 'AQI-IN', 'sens...",2021-12-02 10:50:00,650,95,99,1
443,3,3,266881285112,9,29.863043,77.898801,Outdoor 4,https://secure.aqi.in/public/hwimages/Monitor(...,outdoor,office,Office Room,NaN,NaN,Roorkee,Uttarakhand,India,247667.0,0,2021-11-25 03:11:40 AM,Asia/Calcutta,"[{'sensorid': 1, 'sensorname': 'AQI-IN', 'sens...",2021-12-02 10:50:00,650,124,139,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1360,18,18,162951285014,9,29.863063,77.898746,Outdoor 18,https://secure.aqi.in/public/hwimages/Monitor(...,outdoor,office,Office Room,Civil Engineering Department,IIT Roorkee,Rurki,Uttarakhand,India,247667.0,0,2021-11-25 04:09:02 AM,Asia/Calcutta,"[{'sensorid': 1, 'sensorname': 'AQI-IN', 'sens...",2021-12-02 12:15:00,735,98,102,14
1361,19,19,1068968482,12,NaN,NaN,Indoor 1,https://secure.aqi.in/public/hwimages/hw12.png,indoor,office,Office Room,NaN,NaN,Roorkee,Uttarakhand,India,NaN,1,2021-12-02 12:15:45 PM,Asia/Calcutta,"[{'sensorid': 1, 'sensorname': 'AQI-IN', 'sens...",2021-12-02 12:15:00,735,137,157,14
1363,21,21,362141259613,22,29.863221,77.898577,Outdoor 20,NaN,outdoor,office,Office Room,TRANSPORTATION ENGINEERING,CIVIL ENGINEERING DEPARTMENT,Roorkee,Uttarakhand,India,247667.0,0,2021-11-15 06:15:40 PM,Asia/Calcutta,"[{'sensorid': 1, 'sensorname': 'AQI-IN', 'sens...",2021-12-02 12:15:00,735,52,54,14
1352,10,10,563471285112,9,29.863031,77.898796,Outdoor 11,https://secure.aqi.in/public/hwimages/Monitor(...,outdoor,office,Office Room,NaN,NaN,Delhi,Delhi,India,NaN,0,2021-11-25 06:18:59 AM,Asia/Calcutta,"[{'sensorid': 1, 'sensorname': 'AQI-IN', 'sens...",2021-12-02 12:15:00,735,94,106,14


In [ ]:
heat_df = result_df[["devicelat", "devicelon", "Time","PM 2.5","minute","five_min"]].copy()
heat_df.dropna(subset=['devicelat'], inplace=True)
display(heat_df)

,devicelat,devicelon,Time,PM 2.5,minute,five_min
446,29.863076,77.898693,2021-12-02 10:50:00,119,650,1
440,29.862951,77.898898,2021-12-02 10:50:00,94,650,1
441,29.862980,77.898718,2021-12-02 10:50:00,87,650,1
442,29.863096,77.898811,2021-12-02 10:50:00,95,650,1
443,29.863043,77.898801,2021-12-02 10:50:00,124,650,1
...,...,...,...,...,...,...
1359,29.863034,77.898778,2021-12-02 12:15:00,80,735,14
1360,29.863063,77.898746,2021-12-02 12:15:00,98,735,14
1363,29.863221,77.898577,2021-12-02 12:15:00,52,735,14
1352,29.863031,77.898796,2021-12-02 12:15:00,94,735,14


In [ ]:
def generateBaseMap(default_location=[29.86301, 77.89883], default_zoom_start=14):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map
base_map = generateBaseMap()

In [ ]:
from folium.plugins import HeatMap
base_map = generateBaseMap()
HeatMap(data=heat_df[['devicelat', 'devicelon', 'PM 2.5']].groupby(['devicelat', 'devicelon']).
        sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)
base_map

In [ ]:
from folium.plugins import HeatMapWithTime
df_min_list = []
for minute in heat_df.minute.sort_values().unique():
    df_min_list.append(heat_df.loc[heat_df.minute == minute, ['devicelat', 'devicelon', 'PM 2.5']]
                       .groupby(['devicelat', 'devicelon']).sum().reset_index().values.tolist())
base_map = generateBaseMap(default_zoom_start=14)
HeatMapWithTime(df_min_list, radius=10,  min_opacity=0.5, max_opacity=0.8, use_local_extrema=True,auto_play=True, 
                display_index=True, index_steps=1, min_speed=1, max_speed=10, speed_step=1, position='bottomleft',
                overlay=True, control=True, show=True).add_to(base_map)
base_map

In [ ]:
from folium.plugins import HeatMapWithTime
df_min_list = []
for five_min in heat_df.five_min.sort_values().unique():
    df_min_list.append(heat_df.loc[heat_df.five_min == five_min, ['devicelat', 'devicelon', 'PM 2.5']].groupby(['devicelat', 'devicelon']).sum().reset_index().values.tolist())
base_map = generateBaseMap(default_zoom_start=14)
HeatMapWithTime(df_min_list, radius=10,  min_opacity=0.5, max_opacity=0.8, use_local_extrema=True,auto_play=True, display_index=True, index_steps=1, min_speed=1, max_speed=10, speed_step=1, position='bottomleft', overlay=True, control=True, show=True).add_to(base_map)
base_map